<header style="width:100%;position:relative">
  <div style="width:80%;float:right;">
    <h1>Challenge Loan Approval Prediction in Pennsylvania</h1>
    <h3>Carga y limpieza de los datos</h3>
    <h5>Grupo 2</h5>
  </div>
        <img style="width:15%;" src="./images/logo.jpg" alt="UPM" />
</header>


# Índice

1. [Importar librerias](#1.-Importar-librerias)  
2. [Funciones auxiliares](#2.-Funciones-auxiliares)  
3. [Carga del Dataset](#3.-Carga-del-Dataset)  
4. [Limpieza de los datos](#4.-Limpieza-de-los-datos)  
   * 4.1 [Estudio de las columnas](##4.1-Estudio-de-las-columnas)
   * 4.2 [Tratamiento de los duplicados y de los nulos](##4.2-Tratamiento-de-los-duplicados-y-de-los-nulos)
   * 4.3 [Tratamiento de las variables categóricas](##4.3-Tratamiento-de-las-variables-categoricas)
      * 4.3.1 [Tratamiento de las variables categóricas binarias](###4.3.1-Tratamiento-de-las-variables-categoricas-binarias)
      * 4.3.2 [Tratamiento de las variables categóricas no binarias](###4.3.2-Tratamiento-de-las-variables-categoricas-no-binarias)
5. [Correlacion](#5.-Correlacion)  
6. [Codificacion One-Hot](#6.-Codificacion-One-Hot)  
7. [Create Test](#7.-Create-Test)  
8. [Exportar CSV](#8.-Exportar-CSV)
9. [Referencias](#9.-Referencias)


# 1. Importar librerias

In [69]:
import pandas as pd
import numpy as np

from datetime import datetime
import json


import matplotlib.pyplot as plt
import seaborn as sns


import spacy
from spacy import displacy


# 2. Funciones auxiliares

In [70]:
def extract_month(date, mode=0):
    if pd.isna(date):
        return mode
    return datetime.strptime(date, '%d-%b-%y').month

In [71]:
def month_to_quarter(month):
    if month == 0:
        return 0
    return np.ceil(month / 3).astype(int)

In [72]:
def print_counts(dataframe_function):
    
    value_counts = dataframe_function.value_counts()
    sorted_values = sorted(value_counts.index)
    print("value_counts", ", ".join(sorted_values))
    print("\nNúmero de apariciones:\n", value_counts)


# 3. Carga del Dataset

In [73]:
url = "data/train.csv" 
data = pd.read_csv(url)
df = data.copy()

url2 = "data/test_nolabel.csv"
df_test = pd.read_csv(url2)

Configuramos Pandas para mostrar todas las columnas de un DataFrame al imprimirlo, y luego mostramos las primeras filas del DataFrame df.

In [74]:
pd.set_option('display.max_columns', None)
df.head()

,id,label,statement,subject,speaker,speaker_job,state_info,party_affiliation
0,81f884c64a7,1,China is in the South China Sea and (building)...,"china,foreign-policy,military",donald-trump,President-Elect,New York,republican
1,30c2723a188,0,With the resources it takes to execute just ov...,health-care,chris-dodd,U.S. senator,Connecticut,democrat
2,6936b216e5d,0,The (Wisconsin) governor has proposed tax give...,"corporations,pundits,taxes,abc-news-week",donna-brazile,Political commentator,"Washington, D.C.",democrat
3,b5cd9195738,1,Says her representation of an ex-boyfriend who...,"candidates-biography,children,ethics,families,...",rebecca-bradley,NaN,NaN,none
4,84f8dac7737,0,At protests in Wisconsin against proposed coll...,"health-care,labor,state-budget",republican-party-wisconsin,NaN,Wisconsin,republican


Con *Info* desplegamos la información del dataframe para saber con que datos trabajaremos en cada una de las columnas

In [75]:
df.info

<bound method DataFrame.info of                id  label                                          statement  \
0     81f884c64a7      1  China is in the South China Sea and (building)...   
1     30c2723a188      0  With the resources it takes to execute just ov...   
2     6936b216e5d      0  The (Wisconsin) governor has proposed tax give...   
3     b5cd9195738      1  Says her representation of an ex-boyfriend who...   
4     84f8dac7737      0  At protests in Wisconsin against proposed coll...   
...           ...    ...                                                ...   
8945  44edff2b865      1  If Rhode Island does a hybrid [retirement] pla...   
8946  4a63b5f9c16      1  The new health care law will force seniors int...   
8947  7c57fa8e81c      0  The health insurance plan that (members of Con...   
8948  2375e3cf4b7      1  No one in American history has moved from a Ju...   
8949  5ae9b14e6e5      0  Says the Army is spending $7 million to sponso...   

                   

Gracias a *shape* podemos saber las dimensiones del dataframe, de esta maenra podemos saber la cantidad de filas que tenemos en un incio y la cantidad de columnas con las que trabajaremos

In [76]:
df.shape

(8950, 8)

Visualización del nombre de todas las columnas para plantear la limpieza

In [77]:
df.columns

Index(['id', 'label', 'statement', 'subject', 'speaker', 'speaker_job',
       'state_info', 'party_affiliation'],
      dtype='object')


Visualización del estadisticos de las columnas numericas para plantear la limpieza

# 4. Limpieza de los datos

## 4.1 Estudio de las columnas

This dataset is related to false political claim detection, containing labeled statements (or claims) along with metadata about the speaker, subject, and other contextual information. Below is a detailed breakdown of the dataset's structure and key characteristics:

| **Field Name**      | **Data Type** | **Description**                                                                 |
|----------------------|---------------|---------------------------------------------------------------------------------|
| **id**              | Text          | Unique identifier                                                              |
| **label**           | Integer       | Truthfulness score. Binary classification: 1 = true, 0 = false                |
| **statement**       | Text          | Text of the claim                                                              |
| **subject**         | Text          | Topic of the claim                                                             |
| **speaker**         | Text          | Person making the claim                                                        |
| **speaker_job**     | Text          | Occupation of the speaker                                                      |
| **state_info**      | Text          | Geographic context                                                             |
| **party_affiliation** | Text        | Political party of the speaker                                                 |


Visualización de los tipos de cada columna

In [78]:
df.dtypes

id                   object
label                 int64
statement            object
subject              object
speaker              object
speaker_job          object
state_info           object
party_affiliation    object
dtype: object

Visualizacón de valores distintos que tienen cada columna

In [79]:
# Obtener el número de valores distintos de cada columna
valores_distintos = {columna: df[columna].nunique() for columna in df.columns}

# Mostrar el número de valores distintos de cada columna
for columna, num_valores in valores_distintos.items():
    print(f"Columna: {columna}, Valores distintos: {num_valores}")

Columna: id, Valores distintos: 8950
Columna: label, Valores distintos: 2
Columna: statement, Valores distintos: 8939
Columna: subject, Valores distintos: 3409
Columna: speaker, Valores distintos: 2634
Columna: speaker_job, Valores distintos: 1090
Columna: state_info, Valores distintos: 78
Columna: party_affiliation, Valores distintos: 24


Eliminar id al ser información no necesaria

In [80]:
dfc = df.drop(columns='id')
dfc.head()

,label,statement,subject,speaker,speaker_job,state_info,party_affiliation
0,1,China is in the South China Sea and (building)...,"china,foreign-policy,military",donald-trump,President-Elect,New York,republican
1,0,With the resources it takes to execute just ov...,health-care,chris-dodd,U.S. senator,Connecticut,democrat
2,0,The (Wisconsin) governor has proposed tax give...,"corporations,pundits,taxes,abc-news-week",donna-brazile,Political commentator,"Washington, D.C.",democrat
3,1,Says her representation of an ex-boyfriend who...,"candidates-biography,children,ethics,families,...",rebecca-bradley,NaN,NaN,none
4,0,At protests in Wisconsin against proposed coll...,"health-care,labor,state-budget",republican-party-wisconsin,NaN,Wisconsin,republican


## 4.2 Tratamiento de los duplicados y de los nulos

Comprobación de duplicados

In [81]:
print(dfc.duplicated().any())

False


No hay duplicados

Comporbación de si existen nulos y en que columnas se encuentran

In [82]:
dfc.isnull().sum() 

label                   0
statement               0
subject                 0
speaker                 0
speaker_job          2482
state_info           1930
party_affiliation       0
dtype: int64

Al ser demasiados nulos intentaremos sacar la información de otras columnas

## 4.3 Tratamiento features

### 4.3.1 Features

- Statement

In [83]:
# dfc['Statement']


#### 1. Limipieza y Normalización del Texto 

#### 1.1 Eliminar Ruido
Quitar URLs, etiquetas HTML, menciones, hashtags y caracteres no alfanúmericos

#### 1.2 Convertir a minúsculas

### 4.3.2 Resto de features


- subject

In [84]:
# dfc['subject'].value_counts()

dfc['subject'].iloc[1:10]                    

1                                          health-care
2             corporations,pundits,taxes,abc-news-week
3    candidates-biography,children,ethics,families,...
4                       health-care,labor,state-budget
5                                 candidates-biography
6                     legal-issues,state-budget,states
7                                                 iraq
8                                          immigration
9     economy,income,retirement,social-security,wealth
Name: subject, dtype: object

In [85]:
dfc['subject_list'] = (
    dfc['subject']
    .str.split(',')
    .apply(lambda lst: [s.strip() for s in lst if s.strip()])
)

# 3. “Explode” para tener una fila por cada etiqueta y contar
subjects_exploded = dfc.explode('subject_list')
subject_counts = subjects_exploded['subject_list'].value_counts()
print(subject_counts)
# 4. Convertir a diccionario
# subject_count_dict = subject_counts.to_dict()
# with open('json-study/subject_counts.json', 'w', encoding='utf-8') as f:
#     json.dump(subject_count_dict, f, ensure_ascii=False, indent=2)


subject_list
economy           997
health-care       991
taxes             857
federal-budget    646
education         638
                 ... 
after-the-fact      1
nightlife           1
food                1
homeless            1
death-penalty       1
Name: count, Length: 142, dtype: int64


Vamos a cortar el diccionario por el 1% de los datos para evitar posible overfiting

In [86]:
percentage = 0.01
num_filas = dfc.shape[0]
cut_count_namber = int(num_filas * percentage)
print("Número de filas para el corte:", cut_count_namber)

Número de filas para el corte: 89


In [87]:
from sklearn.preprocessing import MultiLabelBinarizer
df2 = dfc.copy()

frequent_subjects = set(subject_counts[subject_counts >= cut_count_namber].index)

# Sustituir las etiquetas raras por 'other_subject'
df2['subject_list_clean'] = df2['subject_list'].apply(
    lambda lst: [s if s in frequent_subjects else 'other_topics' for s in lst]
)

# Binariza con MultiLabelBinarizer
mlb = MultiLabelBinarizer()
onehot = mlb.fit_transform(df2['subject_list_clean'])

# DataFrame de one-hot y concatenación
df_subjects_onehot = pd.DataFrame(
    onehot,
    columns=mlb.classes_,
    index=df2.index
)

df2 = pd.concat([df2, df_subjects_onehot], axis=1)

# df2.head()
df2.shape

(8950, 71)

- Speaker

In [88]:
# 1. Normalizar
dfc['speaker'] = dfc['speaker'].str.lower().str.strip()

# Obtener el conteo de valores únicos de 'speaker'
speaker_counts = dfc['speaker'].value_counts()

# Convertir a diccionario
speaker_counts_dict = speaker_counts.to_dict()

# Exportar a un archivo JSON
with open('json-study/speaker_counts.json', 'w', encoding='utf-8') as f:
    json.dump(speaker_counts_dict, f, ensure_ascii=False, indent=2)


# Elegimos por donde cortar
top_n = 10
frequent_speakers = set(speaker_counts.head(top_n).index)

# 4. Agrupar los menos frecuentes como 'other_speakers'
dfc['speaker_grouped'] = dfc['speaker'].apply(
    lambda s: s if s in frequent_speakers else 'other_speakers'
)

dfc['speaker_grouped'].value_counts()

speaker_grouped
other_speakers     7211
barack-obama        435
donald-trump        247
hillary-clinton     204
mitt-romney         142
john-mccain         139
chain-email         128
scott-walker        123
rick-perry          117
rick-scott          105
marco-rubio          99
Name: count, dtype: int64

Utilizando los archivos JSON json-study/speaker_counts_test.json y json-study/speaker_counts_test.json, que contienen los conteos de apariciones de cada speaker tanto en el conjunto de entrenamiento como en el de prueba, se ha obtenido la lista de los 20 speakers más frecuentes en el conjunto de prueba.

Al analizar dicha lista, se observa que a partir del décimo speaker se produce una disminución significativa en la frecuencia de aparición. Por tanto, este punto podría considerarse un umbral razonable para realizar un corte: mantener los 10 speakers más frecuentes como categorías individuales y agrupar el resto bajo una categoría común, como "other_speakers". Esta decisión busca reducir el riesgo de overfitting y mejorar la capacidad de generalización del modelo.

<small>

| ID  | Name               | dfc | df_test |
|-----|--------------------|-----|---------|
| 1   | barack-obama       | 435 | 176     |
| 2   | donald-trump       | 247 | 96      |
| 3   | hillary-clinton    | 204 | 93      |
| 4   | mitt-romney        | 142 | 70      |
| 5   | john-mccain        | 139 | 50      |
| 6   | chain-email        | 128 | 50      |
| 7   | scott-walker       | 123 | 60      |
| 8   | rick-perry         | 117 | 56      |
| 9   | rick-scott         | 105 | 45      |
| 10  | marco-rubio        | 99  | 54      |
| 11  | ted-cruz           | 82  | 36      |
| 12  | bernie-s           | 75  | 31      |
| 13  | facebook-posts     | 75  | 25      |
| 14  | chris-christie     | 74  | 30      |
| 15  | charlie-crist      | 62  | 28      |
| 16  | newt-gingrich      | 62  | 21      |
| 17  | jeb-bush           | 60  | 19      |
| 18  | blog-posting       | 57  | 24      |
| 19  | joe-biden          | 54  | 22      |
| 20  | paul-ryan          | 50  | 20      |

</small>


- speaker_job

In [89]:
# 1. Normalizar los textos de 'speaker_job'
dfc['speaker_job'] = (
    dfc['speaker_job']
    .str.lower()
    .str.strip()
    .str.replace(r'\s+', '_', regex=True)
)

# 2. Obtener el conteo de valores únicos de 'speaker_job'
speaker_job_counts = dfc['speaker_job'].value_counts()

# 3. Convertir a diccionario
speaker_job_counts_dict = speaker_job_counts.to_dict()

# 4. Exportar a un archivo JSON
with open('json-study/speaker_job_counts.json', 'w', encoding='utf-8') as f:
    json.dump(speaker_job_counts_dict, f, ensure_ascii=False, indent=2)

# 5. Elegir el punto de corte (por ejemplo, los 10 más frecuentes)
top_n = 13
frequent_speaker_jobs = set(speaker_job_counts.head(top_n).index)

# 6. Agrupar los menos frecuentes como 'other_speaker_jobs'
dfc['speaker_job_grouped'] = dfc['speaker_job'].apply(
    lambda job: job if job in frequent_speaker_jobs else 'other_speaker_jobs'
)

# 7. Ver el resultado final
dfc['speaker_job_grouped'].value_counts()


speaker_job_grouped
other_speaker_jobs               5883
u.s._senator                      627
president                         438
governor                          368
u.s._representative               260
president-elect                   247
presidential_candidate            216
state_senator                     186
state_representative              155
former_governor                   143
senator                           129
milwaukee_county_executive        123
attorney                           94
u.s._house_of_representatives      81
Name: count, dtype: int64

<small>

| id  | speaker_job                                | train_count | test_count |
|-----|--------------------------------------------|-------------|------------|
| 1   | u_s__senator                               | 627         | 204        |
| 2   | president                                  | 438         | 177        |
| 3   | governor                                   | 368         | 152        |
| 4   | u_s__representative                        | 260         | 61         |
| 5   | president-elect                            | 247         | 96         |
| 6   | presidential_candidate                     | 216         | 99         |
| 7   | state_senator                              | 186         | 36         |
| 8   | state_representative                       | 155         | 30         |
| 9   | former_governor                            | 143         | 70         |
| 10  | senator                                    | 129         | 64         |
| 11  | milwaukee_county_executive                 | 123         | 60         |
| 12  | attorney                                   | 94          | 32         |
| 13  | u_s__house_of_representatives              | 81          | 37         |
| 14  | social_media_posting                       | 75          | 25         |
| 15  | governor_of_new_jersey                     | 74          | 30         |
| 16  | congressman                                | 70          | 38         |
| 17  | co-host_on_cnn_s__crossfire                | 66          | 22         |
| 18  | u_s__congressman                           | 56          | 26         |
| 19  | speaker_of_the_house_of_representatives    | 49          | 23         |
| 20  | congresswoman                              | 49          | 18         |

</small>


Utilizando los archivos JSON json-study/speaker__job_counts_test.json y json-study/speaker_job_counts_test.json, que contienen los conteos de apariciones de cada speaker_job tanto en el conjunto de entrenamiento como en el de prueba, se ha obtenido la lista de los 20 speakers_jobs más frecuentes en el conjunto de prueba.

Al analizar dicha lista, se observa que a partir del treceavo speaker_job se produce una disminución significativa en la frecuencia de aparición. Por tanto, este punto podría considerarse un umbral razonable para realizar un corte: mantener los 13 speakers_jobs más frecuentes como categorías individuales y agrupar el resto bajo una categoría común, como "other_speakers_job". Esta decisión busca reducir el riesgo de overfitting y mejorar la capacidad de generalización del modelo.

- state_info

In [90]:
# 1. Obtener el conteo de valores únicos de 'state_info'
state_info_counts = dfc['state_info'].value_counts()

# 2. Convertir a diccionario
state_info_counts_dict = state_info_counts.to_dict()

# 3. Exportar a un archivo JSON
with open('json-study/state_info_counts.json', 'w', encoding='utf-8') as f:
    json.dump(state_info_counts_dict, f, ensure_ascii=False, indent=2)

# 4. Elegir el punto de corte (por ejemplo, los 10 más frecuentes)
top_n = 15
frequent_state_info = set(state_info_counts.head(top_n).index)

# 5. Agrupar los menos frecuentes como 'other_state_info'
dfc['state_info_grouped'] = dfc['state_info'].apply(
    lambda info: info if info in frequent_state_info else 'other_state_info'
)

# 6. Ver el resultado final
dfc['state_info_grouped'].value_counts()


state_info_grouped
other_state_info    3064
Texas                879
Florida              853
Wisconsin            648
New York             579
Illinois             487
Ohio                 408
Georgia              381
Virginia             368
Rhode Island         317
Oregon               220
New Jersey           209
Massachusetts        167
Arizona              160
California           121
Washington, D.C.      89
Name: count, dtype: int64

| Número | Estado               | Train | Test |
|--------|----------------------|-------|------|
| 1      | Texas                | 879   | 380  |
| 2      | Florida              | 853   | 378  |
| 3      | Wisconsin            | 648   | 252  |
| 4      | New York             | 579   | 250  |
| 5      | Illinois             | 487   | 205  |
| 6      | Ohio                 | 408   | 178  |
| 7      | Georgia              | 381   | 164  |
| 8      | Virginia             | 368   | 144  |
| 9      | Rhode Island         | 317   | 135  |
| 10     | New Jersey           | 209   | 96   |
| 11     | Oregon               | 220   | 87   |
| 12     | Massachusetts        | 167   | 83   |
| 13     | Arizona              | 160   | 68   |
| 14     | California           | 121   | 59   |
| 15     | Washington, D.C.     | 89    | 52   |
| 16     | Vermont              | 80    | 37   |
| 17     | New Hampshire        | 79    | 28   |
| 18     | Pennsylvania         | 79    | 33   |
| 19     | Arkansas             | 77    | 26   |
| 20     | Kentucky             | 72    | 26   |


Utilizando los archivos JSON json-study/state_info_counts_test.json y json-study/state_info_counts.json, que contienen los conteos de apariciones de cada estado tanto en el conjunto de entrenamiento como en el de prueba, se ha obtenido la lista de los 20 estados más frecuentes en el conjunto de prueba.

Al analizar dicha lista, se observa que a partir del quinceavo estado se produce una disminución significativa en la frecuencia de aparición. Por tanto, este punto podría considerarse un umbral razonable para realizar un corte: mantener los 15 estados más frecuentes como categorías individuales y agrupar el resto bajo una categoría común, como "other_states". Esta decisión busca reducir el riesgo de overfitting y mejorar la capacidad de generalización del modelo.

- party_affiliation

In [91]:
# dfc['party_affiliation'].value_counts()       

# Obtener el conteo de valores únicos de 'speaker'
party_affiliation_counts = dfc['party_affiliation'].value_counts()

# Convertir a diccionario
party_affiliation_counts_dict = party_affiliation_counts.to_dict()

# Exportar a un archivo JSON
with open('json-study/party_affiliation_counts.json', 'w', encoding='utf-8') as f:
    json.dump(party_affiliation_counts_dict, f, ensure_ascii=False, indent=2)


In [92]:
mapping = {
    # Major parties
    'republican': 'Republican',
    'democrat': 'Democrat',
    'democratic-farmer-labor': 'Democrat',

    # Independents / None
    'none': 'Independent_None',
    'independent': 'Independent_None',
    'newsmaker': 'Independent_None',
    'journalist': 'Independent_None',
    'columnist': 'Independent_None',
    'activist': 'Independent_None',
    'talk-show-host': 'Independent_None',

    # Third parties
    'libertarian': 'Third_Party',
    'green': 'Third_Party',
    'constitution-party': 'Third_Party',
    'liberal-party-canada': 'Third_Party',

    # Organizations
    'organization': 'Organization',
    
    # Officials  
    'state-official': 'Official',
    'business-leader': 'Official',
    'labor-leader': 'Official',
    'education-official': 'Official',
    'government-body': 'Official',

    # Tea Party
    'tea-party-member': 'Tea_Party',
    'ocean-state-tea-party-action': 'Tea_Party',
}

def group_party(x):
    return mapping.get(x, 'Other')  # anything else → 'Other'

df['party_group'] = df['party_affiliation'].apply(group_party)

# Recuento de las nuevas categorías
print(df['party_group'].value_counts())


party_group
Republican          3947
Democrat            2899
Independent_None    1824
Organization         197
Third_Party           40
Official              32
Tea_Party              9
Other                  2
Name: count, dtype: int64


# 7. Create Test

In [93]:
dfa = df_test.copy() # Backup del dataframe original
dfa.dtypes.sort_index()

id                   object
party_affiliation    object
speaker              object
speaker_job          object
state_info           object
statement            object
subject              object
dtype: object

In [94]:
# Opcional: Eliminar filas con valores nulos en el DataFrame 'df_test'
# df_test = df_test.dropna()

# # Mostrar la cantidad de valores nulos restantes en 'df_test'
print(df_test.isnull().sum())

id                      0
statement               0
subject                 0
speaker                 0
speaker_job          1082
state_info            818
party_affiliation       0
dtype: int64


- Speaker

In [95]:
# Obtener el conteo de valores únicos de 'speaker'
speaker_counts = dfa['speaker'].value_counts()

# Convertir a diccionario
speaker_counts_dict = speaker_counts.to_dict()

# Exportar a un archivo JSON
with open('json-study/speaker_counts_test.json', 'w', encoding='utf-8') as f:
    json.dump(speaker_counts_dict, f, ensure_ascii=False, indent=2)

- speaker_job

In [96]:
# Obtener el conteo de valores únicos de 'speaker'
speaker_job_counts = dfa['speaker_job'].value_counts()

# Convertir a diccionario
speaker_job_counts_dict = speaker_job_counts.to_dict()

# Exportar a un archivo JSON
with open('json-study/speaker_job_counts_test.json', 'w', encoding='utf-8') as f:
    json.dump(speaker_job_counts_dict, f, ensure_ascii=False, indent=2)

- state_info

In [97]:
# Obtener el conteo de valores únicos de 'speaker'
state_info_counts = dfa['state_info'].value_counts()

# Convertir a diccionario
state_info_counts_dict = state_info_counts.to_dict()

# Exportar a un archivo JSON
with open('json-study/state_info_counts_test.json', 'w', encoding='utf-8') as f:
    json.dump(state_info_counts_dict, f, ensure_ascii=False, indent=2)

- party_affiliation

In [98]:
# dfc['party_affiliation'].value_counts()       

# Obtener el conteo de valores únicos de 'speaker'
party_affiliation_counts = dfa['party_affiliation'].value_counts()

# Convertir a diccionario
party_affiliation_counts_dict = party_affiliation_counts.to_dict()

# Exportar a un archivo JSON
with open('json-study/party_affiliation_counts_test.json', 'w', encoding='utf-8') as f:
    json.dump(party_affiliation_counts_dict, f, ensure_ascii=False, indent=2)


# 8. Exportar CSV

Comparar los dos dataset para ver que todo esta bien

In [99]:
df_check =  pd.read_csv("data/test_nolabel.csv")

In [100]:
# Comparar las columnas de dfc y df_testn
columns_dfc = set(dfc.columns)
columns_df_testn = set(df_testn.columns)

# Encontrar las diferencias
differences = columns_dfc.symmetric_difference(columns_df_testn)

# Verificar si la única diferencia es 'id'
if differences == {'id', 'Accept'}:
    print("La única diferencia entre las columnas es 'id' y 'Accept'.")
else:
    print("Existen otras diferencias en las columnas:", differences)

NameError: name 'df_testn' is not defined

Guardar en *formated* los csv train y test

In [ ]:
# Exportar el DataFrame 'dfn' a un archivo CSV
dfc.to_csv('./formated/train_exportado_v2.csv', index=False)

df_testn.to_csv('./formated/test_exportado_v2.csv', index=False)

Todas las features:

In [ ]:
all_features = [
    # Características numéricas
    'ApprovalFY', 'NoEmp', 'CreateJob', 'RetainedJob', 'DisbursementGross',

    # Características categóricas binarias
    'NewExist_Binary', 'Franchise_Binary', 'UrbanRural_Binary',
    'RevLineCr_Binary', 'LowDoc_Binary', 'CreateJob_Binary',
    'RetainedJob_Binary',

    # Características categóricas codificadas
    'ApprovalFY_Grouped_cod', 'NoEmp_Grouped_cod', 
    'DisbursementGross_Grouped_cod',

    # Características temporales
    'ApprovalDate', 'DisbursementDate', 
    'ApprovalDate_quarter', 'DisbursementDate_quarter',

    # Características categóricas sin codificar
    'Bank_Categorized_cod', 'BankState_Categorized_cod',
    'ApprovalFY_Grouped', 'NoEmp_Grouped', 
    'DisbursementGross_Grouped',

    # Características relacionadas con Bank tras un One Hot Encoding
    'Bank_CAPITAL ONE NATL ASSOC', 'Bank_CITIZENS BANK NATL ASSOC',
    'Bank_COMMUNITY CAP. DEVEL CORP', 'Bank_FIFTH THIRD BANK',
    'Bank_FIRSTMERIT BANK, N.A.', 'Bank_HAMILTON CNTY DEVEL COMPANY IN',
    'Bank_JPMORGAN CHASE BANK NATL ASSOC',
    'Bank_KEYBANK NATIONAL ASSOCIATION', 'Bank_Otros',
    'Bank_PNC BANK, NATIONAL ASSOCIATION',
    'Bank_THE HUNTINGTON NATIONAL BANK',
    'Bank_U.S. BANK NATIONAL ASSOCIATION',
    'Bank_WELLS FARGO BANK NATL ASSOC',

    # Características relacionadas con BankState tras un One Hot Encoding
    'BankState_CA', 'BankState_DE', 'BankState_IL', 'BankState_IN',
    'BankState_OH', 'BankState_Otros', 'BankState_RI', 
    'BankState_SD', 'BankState_VA',
]


# 9. Referencias

* [Alexisbcook. (2023, 21 abril). Categorical variables. Kaggle.](https://www.kaggle.com/code/alexisbcook/categorical-variables)
* [pandas documentation — pandas 2.2.3 documentation. (s. f.).](https://pandas.pydata.org/docs/)